# Reflectance FTIR Spectra

- This Jupyter notebook provides an example workflow for processing reflectance FTIR spectra for thicknesses of olivine and glasses through PyIRoGlass. 

- The Jupyter notebook and data can be accessed here: https://github.com/SarahShi/PyIRoGlass/blob/main/docs/examples/reflectance_ftir/. 

- You need to have the PyIRoGlass PyPi package on your machine once. If you have not done this, please uncomment (remove the #) symbol and run the cell below. 

In [ ]:
#!pip install PyIRoGlass

# Load Python Packages

In [ ]:
# Import packages

import os
import sys
import glob
import numpy as np
import pandas as pd

import PyIRoGlass as pig

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rc, cm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pig.__version__

# Olivine Thicknesses

The workflows for processing olivine and glass spectra are slightly different given variable refractive index and wavenumbers of applicability. We'll walk through each individually. 


In [ ]:
# Change paths to direct to folder with olivine reflectance FTIR spectra

REF_PATH_OL = 'SPECTRA/OL/'

ol_loader = pig.SampleDataLoader(spectrum_path=REF_PATH_OL)
REF_DFS_DICT_OL = ol_loader.load_spectrum_directory(wn_high=2700, wn_low=2100)


Let's look at what a dictionary of reflectance FTIR spectra look like. Samples are identified by their file names and the wavenumber and absorbance data are stored for each spectrum. 

In [ ]:
REF_DFS_DICT_OL

The reflectance index is a key input for this method. In olivine, the refractive index is compositionally dependent. Deer, Howie, and Zussman parameterize this value for olivine in An introduction to the rock-forming minerals. The Reflectance_Index function performs this calculation, inputting forsterite content and returning the reflectance index. 

In [ ]:
# Use DHZ parameterization of olivine reflectance index. 

n_ol = pig.reflectance_index(0.72)

print(n_ol)

We're ready to use the Thickness_Process function now. We input the arguments of the dictionary of spectra, reflectance index, the wavenumber range, and booleans of baseline removal, plotting, and whether the phase is olivine. 

We use the wavenumber range of 2700-2100 cm$^{-1}$ in olivine -- see the paper for more details. We remove a baseline from the spectrum to remove variability existing in the absorbances and to allow the data to be centered around 0. This does not change amplitude. We have the phaseol boolean for specifying whether your phase is olivine or not. 

Thickness_Process function workflow is as follows: 
- Remove baseline to center the data around an absorbance of 0. 
- Apply a Savitzky-Golay filter to smooth the data for single-spikes. 
- Identify extrema.
- Calculate wavenumber differences between extrema.
- Calculate mean and $\sigma$ thickness. 

In [ ]:
FUEGO_THICK = pig.calculate_mean_thickness(REF_DFS_DICT_OL, n=n_ol, wn_high=2700, wn_low=2100, plotting=True, phaseol=True)

The thickness dataframe returned quantifies mean and $\sigma$ thicknesses just from the maxima, minima, and both. We recommend using 'Thickness_M' and 'Thickness_STD', which represent the mean and $\sigma$ thickness across all extrema. 


In [ ]:
FUEGO_THICK

# Glass Thicknesses

Now, let's look at glass. The slight variations exist within reflectance index, wavenumbers of interest, and the phaseol boolean. We use the Nichols and Wysoczanski, 2007 reflectance index of 1.546 for basaltic glass in this example. The wavenumber range is extended to 2850-1700 cm$^{-1}$. See the paper for more details. 

I won't show all the intermediate steps, and will jump forward to the results since the workflows are identical aside from these parameters. 

In [ ]:
# Change paths to direct to folder with glass reflectance FTIR spectra

REF_PATH_GL = 'SPECTRA/GL/'

gl_loader = pig.SampleDataLoader(spectrum_path=REF_PATH_GL)
REF_DFS_DICT_GL = gl_loader.load_spectrum_directory(wn_high=2850, wn_low=1700)

# n=1.546 in the range of 2000-2700 cm^-1 following Nichols and Wysoczanski, 2007 for basaltic glass

n_gl = 1.546

REF_GLASS = pig.calculate_mean_thickness(REF_DFS_DICT_GL, n=n_gl, wn_high=2850, wn_low=1700, plotting=True, phaseol=False)

REF_GLASS
